In [4]:
import json
import tkinter as tk
from tkinter import ttk
import requests
import time
import os
from PIL import Image, ImageTk
from datetime import datetime as dt

In [5]:
# API key is bd5bde105252156348b9cfa726000bde\
# api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key} 
# https://api.openweathermap.org/data/2.5/weather?q={city name},{country code}&appid={API key}
# make current weather app in tkinter using api key and city name or country code if required use tkinter ttk module and use icon for weather that is stored in the folder E:\Movies\Summer Training Project 2023\weather app\weather icons
# use icon for weather that is stored in the folder E:\Movies\Summer Training Project 2023\weather app\weather icons
# if required use oops concept

In [6]:
def download_icons():
    url = "https://openweathermap.org/img/wn/"
    icons = ["01d", "01n", "02d", "02n", "03d", "03n", "04d", "04n", "09d", "09n", "10d", "10n", "11d", "11n", "13d", "13n", "50d", "50n"]
    for icon in icons:
        if icon not in os.listdir("weather icons"):    
            response = requests.get(f"{url}{icon}@2x.png")
            with open(f"weather icons\\{icon}.png", "wb") as file:
                file.write(response.content)
            
if not os.path.exists("weather icons"):
    os.mkdir("weather icons")
    download_icons()
elif len(os.listdir("weather icons")) != 18:
    download_icons()
else:
    pass

timezone = 0

class WeatherApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Weather App")
        self.geometry("6000x4000")
        self.resizable(True, True)
        self.attributes("-topmost", True)
        
        # Load the background image
        global bg_photo
        bg_image = Image.open(r"E:\Movies\Summer Training Project 2023\weather app\bg.png")
        bg_photo = ImageTk.PhotoImage(bg_image)

        # Create a Canvas widget for scrolling
        self.canvas = tk.Canvas(self, bg=self.cget('bg'))
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Create a background image on the Canvas
        self.canvas.create_image(0, 0, anchor=tk.NW, image=bg_photo)

        # Create a scrollbar and associate it with the Canvas widget with both vertical and horizontal movement
        self.scrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.scrollbar2 = ttk.Scrollbar(self, orient=tk.HORIZONTAL, command=self.canvas.xview)
        self.scrollbar2.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas.configure(xscrollcommand=self.scrollbar2.set)
        self.canvas.bind('<MouseWheel>', lambda e: self.canvas.yview_scroll(int(-1*(e.delta/120)), "units"))
        self.canvas.bind('<Shift-MouseWheel>', lambda e: self.canvas.xview_scroll(int(-1*(e.delta/120)), "units"))
        self.canvas.bind('<Configure>', lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))  

        # Create a frame to contain your widgets 
        self.content_frame = tk.Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.content_frame, anchor=tk.NW)

        # Add widgets to the content_frame
        self.create_widgets()
        self.get_weather()
        
        
    def create_widgets(self):
        self.city_label = ttk.Label(self.content_frame, text="City: ", font=("Arial", 15))
        self.city_label.pack(pady=10)
        
        self.city_entry = ttk.Entry(self.content_frame, font=("Arial", 15))
        self.city_entry.pack(pady=10)
        self.city_entry.focus()
        self.city_entry.bind("<Return>", lambda event=None: self.get_weather())
        
        self.search_button = ttk.Button(self.content_frame, text="Search", command=self.get_weather)
        self.search_button.pack(padx=10)
        
        self.weather_label = ttk.Label(self.content_frame, text="Weather: ", font=("Arial", 15))
        self.weather_label.pack(pady=10)
        
        self.temp_label = ttk.Label(self.content_frame, text="Temperature: ", font=("Arial", 15))
        self.temp_label.pack(pady=10)
        
        self.feels_like_label = ttk.Label(self.content_frame, text="Feels Like: ", font=("Arial", 15))
        self.feels_like_label.pack(pady=10)
        
        self.humidity_label = ttk.Label(self.content_frame, text="Humidity: ", font=("Arial", 15))
        self.humidity_label.pack(pady=10)
        
        self.wind_speed_label = ttk.Label(self.content_frame, text="Wind Speed: ", font=("Arial", 15))
        self.wind_speed_label.pack(pady=10)
        
        self.wind_direction_label = ttk.Label(self.content_frame, text="Wind Direction: ", font=("Arial", 15))
        self.wind_direction_label.pack(pady=10)
        
        self.sunrise_label = ttk.Label(self.content_frame, text="Sunrise: ", font=("Arial", 15))
        self.sunrise_label.pack(pady=10)
        
        self.sunset_label = ttk.Label(self.content_frame, text="Sunset: ", font=("Arial", 15))
        self.sunset_label.pack(pady=10)
        
        self.error_label = ttk.Label(self.content_frame, text="", font=("Arial", 15), foreground="red")
        self.error_label.pack(pady=10)
        
        self.update_button = ttk.Button(self.content_frame, text="Update", command=self.get_weather)
        self.update_button.pack(pady=10)
        
        self.current_time_label = ttk.Label(self.content_frame, text="", font=("Arial", 15))
        self.current_time_label.pack(pady=10)
        
        self.icon = ImageTk.PhotoImage(Image.open("weather icons\\01d.png"))
        self.icon_label = ttk.Label(self.content_frame, image=self.icon)
        self.icon_label.pack(pady=10)
        
        self.update_time()    
            
    def get_weather(self):
        global timezone
        try:
            self.error_label.config(text="")
            city = self.city_entry.get()
            api_key = "bd5bde105252156348b9cfa726000bde"
            url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
            response = requests.get(url)
            data = json.loads(response.text)
            weather = data["weather"][0]["main"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            wind_speed = data["wind"]["speed"]
            wind_direction = data["wind"]["deg"]
            timezone = data["timezone"]
            sunrise = time.strftime("%I:%M:%S", time.gmtime(data["sys"]["sunrise"] + float(timezone)))
            sunset = time.strftime("%I:%M:%S", time.gmtime(data["sys"]["sunset"] + float(timezone)))
            self.weather_label.config(text=f"Weather: {weather}")
            self.temp_label.config(text=f"Temperature: {temp}°C")
            self.feels_like_label.config(text=f"Feels Like: {feels_like}°C")
            self.humidity_label.config(text=f"Humidity: {humidity}%")
            self.wind_speed_label.config(text=f"Wind Speed: {wind_speed} km/h")
            self.wind_direction_label.config(text=f"Wind Direction: {wind_direction}°")
            self.sunrise_label.config(text=f"Sunrise: {sunrise}")
            self.sunset_label.config(text=f"Sunset: {sunset}")
            
            if weather == "Clear":
                if time.time() > data["sys"]["sunrise"] + float(timezone) and time.time() < data["sys"]["sunset"] + float(timezone):
                    self.icon = ImageTk.PhotoImage(Image.open("weather icons\\01d.png"))
                else:
                    self.icon = ImageTk.PhotoImage(Image.open("weather icons\\01n.png"))
            elif weather == "Clouds":
                if time.time() > data["sys"]["sunrise"] + float(timezone) and time.time() < data["sys"]["sunset"] + float(timezone):
                    self.icon = ImageTk.PhotoImage(Image.open("weather icons\\02d.png"))
                else:
                    self.icon = ImageTk.PhotoImage(Image.open("weather icons\\02n.png"))
            elif weather == "Rain":
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\10d.png"))
            elif weather == "Thunderstorm":
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\11d.png"))
            elif weather == "Drizzle":
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\09d.png"))
            elif weather == "Snow":
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\13d.png"))
            elif weather == "Mist" or weather == "Smoke" or weather == "Haze" or weather == "Dust" or weather == "Fog" or weather == "Sand" or weather == "Dust" or weather == "Ash" or weather == "Squall" or weather == "Tornado":
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\50d.png"))
            else:
                self.icon = ImageTk.PhotoImage(Image.open("weather icons\\01d.png"))
            
            self.icon_label.config(image=self.icon)
            self.icon_label.pack(pady=10)
            
        except Exception as e:
            self.error_label.config(text=f"Error: {str(e)}")
            self.weather_label.config(text="Weather: ")
            self.temp_label.config(text="Temperature: ")
            self.feels_like_label.config(text="Feels Like: ")
            self.humidity_label.config(text="Humidity: ")
            self.wind_speed_label.config(text="Wind Speed: ")
            self.wind_direction_label.config(text="Wind Direction: ")
            self.sunrise_label.config(text="Sunrise: ")
            self.sunset_label.config(text="Sunset: ")
            self.icon_label.config(image="")
            print(e)
            
    def update_time(self): 
        current_time = time.strftime("%I:%M:%S %p", time.gmtime(time.time() + timezone))
        self.current_time_label.config(text=current_time)
        self.current_time_label.after(1000, self.update_time)
        
if __name__ == "__main__":
    app = WeatherApp()
    app.mainloop()

'weather'
